In [13]:
import torch
from pprint import pprint
from omegaconf import OmegaConf
from IPython.display import Audio, display
from torch import nn

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)
models = OmegaConf.load('latest_silero_models.yml')

In [14]:
language = 'ru'
speaker = 'kseniya_16khz'
device = torch.device('cuda:0')
model, symbols, sample_rate, example_text, apply_tts = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                      model='silero_tts',
                                                                      language=language,
                                                                      speaker=speaker)
model = model.to(device)

Using cache found in /home/jovyan/.cache/torch/hub/snakers4_silero-models_master


In [15]:
print(torch.jit.script(model.tacotron.embedding).code)

def forward(self,
    input: Tensor) -> Tensor:
  _0 = __torch__.torch.nn.functional.embedding
  weight = self.weight
  _1 = _0(input, weight, None, None, 2., False, False, )
  return _1



In [16]:
example_text = '''Октябрь уж наступил — уж роща отряхает Последние листы с нагих своих ветвей; Дохнул осенний хлад — дорога промерзает. Журча еще бежит за мельницу ручей,
.'''

cleaned_example_text = "".join([c if c in symbols else " " for c in example_text])

speaker_id = torch.tensor(0).to(device)
embeder = model.tacotron.embedding.to(device)
encoder = model.tacotron.encoder.to(device)
decoder = model.tacotron.decoder.to(device)

input_tensor = torch.LongTensor([symbols.index(c) for c in cleaned_example_text]).unsqueeze(0).to(device)
embedded_inputs = embeder(input_tensor)
print(embedded_inputs.shape)

embedded_inputs = torch.transpose(embedded_inputs, 1, 2)
encoder_output = encoder(embedded_inputs)
print(encoder_output.shape)

mel_outputs, gate_outputs, alignments, mel_lengths = decoder(encoder_output)

torch.Size([1, 154, 512])


/home/jovyan/venvs/sinara/lib/python3.10/site-packages/torch/nn/modules/module.py:1527: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  return forward_call(*args, **kwargs)


torch.Size([1, 154, 512])


In [17]:
print("mel_outputs", mel_outputs.shape)  # <- use this as speech data representation
print("gate_outputs", gate_outputs.shape)
print("alignments", alignments.shape)
print("mel_lengths", mel_lengths.shape)

mel_outputs torch.Size([1, 128, 1000])
gate_outputs torch.Size([1, 1000])
alignments torch.Size([1, 1000, 154])
mel_lengths torch.Size([1])


In [18]:
output = model.vocoder(mel_outputs)
output

tensor([[[-0.0003, -0.0002, -0.0003,  ...,  0.0281,  0.0668,  0.0304]]],
       device='cuda:0')

In [19]:
import IPython.display as ipd  # type: ignore

def display_audio(samples: torch.Tensor, sample_rate: int):
    """Renders an audio player for the given audio samples.

    Args:
        samples (torch.Tensor): a Tensor of decoded audio samples
            with shapes [B, C, T] or [C, T]
        sample_rate (int): sample rate audio should be displayed with.
    """
    assert samples.dim() == 2 or samples.dim() == 3

    samples = samples.detach().cpu()
    if samples.dim() == 2:
        samples = samples[None, ...]

    for audio in samples:
        ipd.display(ipd.Audio(audio, rate=sample_rate))

In [20]:
output.shape

torch.Size([1, 1, 200000])

In [29]:
display_audio(output.detach().cpu(), sample_rate=18000)

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [33]:
from einops import rearrange

rearrange(outputs[0], 'b n -> b 1 n')

NameError: name 'outputs' is not defined

In [61]:
inputs, labels = input_tensor.to(device), output.to(device)

optimizer.zero_grad()

outputs = model(inputs)
loss = criterion(outputs, labels)
loss.backward()
optimizer.step()

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/___torch_mangle_70.py", line 10, in forward
  def forward(self: __torch__.___torch_mangle_70.TTSModel,
    sequence: Tensor) -> Tuple[Tensor, Tensor]:
    _0 = (self.tacotron).forward(sequence, None, )
          ~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
    mel_outputs_postnet, mel_lengths, = _0
    _1 = (self.vocoder).forward(mel_outputs_postnet, )
  File "code/__torch__/jit_model.py", line 15, in forward
    inputs: Tensor,
    max_decoder_steps: Optional[int]=None) -> Tuple[Tensor, Tensor]:
    embedded_inputs = torch.transpose((self.embedding).forward(inputs, ), 1, 2)
                                       ~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
    encoder_outputs = (self.encoder).forward(embedded_inputs, )
    _0 = (self.decoder).forward(encoder_outputs, max_decoder_steps, )
  File "code/__torch__/torch/nn/modules/sparse.py", line

In [34]:
import pandas as pd

yandex = pd.read_csv("dataset.csv")

In [35]:
print(yandex.iloc[:, 7][0])

Пылью наполнены обе сандалии
В поисках гения по спирали
Хлопьями воздуха манит дорога
Голоса нет, аллергия
Не многого надо

Поцелуй от помады
Побережье Невады
И кофе глясе
Когда тебе рады
Любовь до упада
И дождливое танго!
Так бежим по шоссе!

Это мой нулевой километр!
Пожирает сентябрьский полдень
По колёсам играет оркестр
Камень за камнем, камень за камнем

Это мой нулевой километр!
Начинаю отсчёт от Казани
Мы построим Китайскую стену здесь!
Камень за камнем

Пересекающий ток расстояния
Беспрерывное эхо молчаливых признаний
Я подарю поцелуй и три слова
Голоса нет, аллергия — так многого надо

Простыню и пижаму
Взгляд принцессы Дианы
И запой по весне
Еще два банана
Нежности от коалы
Может просто устали?
Ну, так бежим по шоссе!

Это мой нулевой километр!
Пожирает сентябрьский полдень
По колёсам играет оркестр
Камень за камнем, камень за камнем

Это мой нулевой километр!
Начинаю отсчёт от Казани
Мы построим Китайскую стену здесь!
Камень за камнем, камень за камнем
(Камень за камнем, кам

In [36]:
cleaned_text = "".join([c if c in symbols else " " for c in yandex.iloc[:, 7][0]])
cleaned_text

' ылью наполнены обе сандалии   поисках гения по спирали  лопьями воздуха манит дорога  олоса нет, аллергия  е многого надо   оцелуй от помады  обережье  евады   кофе глясе  огда тебе рады  юбовь до упада   дождливое танго!  ак бежим по шоссе!   то мой нулевой километр!  ожирает сентябрьский полдень  о колёсам играет оркестр  амень за камнем, камень за камнем   то мой нулевой километр!  ачинаю отсчёт от  азани  ы построим  итайскую стену здесь!  амень за камнем   ересекающий ток расстояния  еспрерывное эхо молчаливых признаний   подарю поцелуй и три слова  олоса нет, аллергия   так многого надо   ростыню и пижаму  згляд принцессы  ианы   запой по весне  ще два банана  ежности от коалы  ожет просто устали?  у, так бежим по шоссе!   то мой нулевой километр!  ожирает сентябрьский полдень  о колёсам играет оркестр  амень за камнем, камень за камнем   то мой нулевой километр!  ачинаю отсчёт от  азани  ы построим  итайскую стену здесь!  амень за камнем, камень за камнем   амень за камнем, ка

In [37]:
input_tensor = torch.LongTensor([symbols.index(c) for c in cleaned_text]).unsqueeze(0).to(device)
input_tensor

tensor([[35, 30, 14,  ..., 16,  7, 15]], device='cuda:0')

In [52]:
import librosa

audio, sr = librosa.load("data/vocals/" + yandex['title'][0] + "_vocals.mp3", sr=None)

input_tensor = torch.tensor(audio, dtype=torch.float32)

input_tensor = input_tensor.unsqueeze(0)

In [53]:
input_tensor.shape

torch.Size([1, 9532800])

In [58]:
from torch.utils.data import Dataset, DataLoader

class MusicDataset(Dataset):
    def __init__(self, vocal_transform=None):
        self.vocal_transform = vocal_transform
        self.yandex = pd.read_csv("dataset.csv")

    def __len__(self):
        return len(self.yandex)

    def __getitem__(self, idx):
        cleaned_text = "".join([c if c in symbols else " " for c in self.yandex.iloc[:, 7][0]])
        input_tensor = torch.LongTensor([symbols.index(c) for c in cleaned_text]).unsqueeze(0).to(device)

        audio, sr = librosa.load("data/vocals/" + self.yandex['title'][idx] + "_vocals.mp3", sr=None)
        input_tensor = torch.tensor(audio, dtype=torch.float32)
        input_tensor = input_tensor.unsqueeze(0)

In [60]:
dataset = MusicDataset()

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
dataloader